In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import warnings 
warnings.filterwarnings("ignore")

In [3]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import requests

In [4]:
url_list = []
for i in range(1,42):
    url = f"https://www.flipkart.com/search?q=mobiles&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as={i}"
    url_list.append(url)

In [5]:
url_list

['https://www.flipkart.com/search?q=mobiles&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=1',
 'https://www.flipkart.com/search?q=mobiles&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=2',
 'https://www.flipkart.com/search?q=mobiles&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=3',
 'https://www.flipkart.com/search?q=mobiles&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=4',
 'https://www.flipkart.com/search?q=mobiles&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=5',
 'https://www.flipkart.com/search?q=mobiles&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=6',
 'https://www.flipkart.com/search?q=mobiles&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=7',
 'https://www.flipkart.com/search?q=mobiles&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=8',
 'https://www.flipkart.com/search?q=mobiles&otracker=search&otracker1=se

In [6]:
def create_html_page(url):
    uClient = urlopen(url)
    page_html = uClient.read()
    uClient.close()
    page_soup = BeautifulSoup(page_html, features='html')
    return page_soup

In [9]:
def create_final_data(page_soup):
    data = {"Product Name": [],
        "Sale & Actual Price": [],
        "Review & Ratings": [],
        "Offer": [],
        "Camera": [],
        "Memory": [],
        "Battery": [],
        "Display": [],
        "Ratings Number": []}
    
    all_info_class_name = "yKfJKb row"
    containers = page_soup.find_all("div", {"class": all_info_class_name})
    
    for contain in containers:
        product_name = contain.find_all("div", {"class": "KzDlHZ"})[0].text
        data["Product Name"].append(product_name)

        rr = contain.find_all("span", {"class": "Wphh3N"})
        if len(rr) == 0:
            review_and_rating = None
            rating_number = None
        else:
            review_and_rating = rr[0].text
            rating_number = contain.find_all("div", {"class": "XQDdHH"})[0].text

        data["Review & Ratings"].append(review_and_rating)
        data["Ratings Number"].append(rating_number)

        specifictations = contain.find_all("div", {"class": "_6NESgJ"})[0]

        data["Memory"].append(specifictations.find_all('li')[0].text)
        data["Display"].append(specifictations.find_all('li')[1].text)
        data["Camera"].append(specifictations.find_all('li')[2].text)
        data["Battery"].append(specifictations.find_all('li')[3].text)

        price = contain.find_all("div", {"class": "hl05eU"})[0].text
        data["Sale & Actual Price"].append(price)

        offer_list = contain.find_all("div", {"class": "UkUFwK"})
        if len(offer_list) == 0:
            data["Offer"].append("No Offer")
        else:
            data["Offer"].append(offer_list[0].text)
        
    return data
    
 




# MAIN SECTION

In [48]:
data_frame_list = []
for i, url in enumerate(url_list):
    print(f"Started url page {i+1}")
    page_soup = create_html_page(url_list[0])
    data_info = create_final_data(page_soup)
    data_frame_list.append(data_info)

Started url page 1
Started url page 2
Started url page 3
Started url page 4
Started url page 5
Started url page 6
Started url page 7
Started url page 8


HTTPError: HTTP Error 429: Too Many Requests

In [49]:
len(data_frame_list)

7

In [50]:
df_all = []
for i in range(len(data_frame_list)):
    df = pd.DataFrame(data_frame_list[i])
    df_all.append(df)

In [51]:
final_df = pd.concat(df_all, ignore_index=True)

In [52]:
final_df["Product Name"].unique().shape

(24,)

In [53]:
final_df["Product Name"].unique()

array(['REDMI Note 14 SE 5G (Crimson Art, 128 GB)',
       'REDMI Note 14 SE 5G (Mystique White, 128 GB)',
       'Samsung Guru Music 2 B310ED', 'POCO C71 (Cool Blue, 128 GB)',
       'REDMI Note 14 SE 5G (Titan Black, 128 GB)',
       'MOTOROLA g35 5G (Guava Red, 128 GB)',
       'MOTOROLA g05 (Forest Green, 64 GB)',
       'POCO C71 (Power Black, 128 GB)',
       'Motorola g45 5G (Brilliant Green, 128 GB)',
       'POCO M6 Plus 5G (Ice Silver, 128 GB)',
       'Samsung Galaxy F05 (Twilight Blue, 64 GB)',
       'POCO C71 (Cool Blue, 64 GB)', 'MOTOROLA g05 (Plum Red, 64 GB)',
       'POCO C71 (Desert Gold, 128 GB)',
       'REDMI A5 (Jaisalmer Gold, 128 GB)', 'Samsung Guru 1200',
       'vivo T4x 5G (Marine Blue, 128 GB)',
       'OPPO K13 5G with 7000mAh and 80W SUPERVOOC Charger In-The-Box (Prism Black, 128 GB)',
       'vivo T4x 5G (Pronto Purple, 256 GB)',
       'POCO C75 5G (Silver Stardust, 64 GB)',
       'vivo T4 Lite 5G Charger in the Box (Titanium Gold, 128 GB)',
       'Mo

In [54]:
final_df.shape

(168, 9)

In [56]:
final_df.to_csv("flipkart_mobile_phones.csv")